# **makemore** character level language model

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt', 'r').read().splitlines()
words[:5]

['emma', 'olivia', 'ava', 'isabella', 'sophia']

In [3]:
len(words)

32033

### (re-)building our training dataset

In [4]:
# create the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [5]:
# build the dataset

block_size = 3
X, Y = [], []

for w in words[:3]:
    print(w)
    context = [0] * block_size
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '----->', itos[ix])
        context = context[1:] + [ix]

X = torch.tensor(X)
Y = torch.tensor(Y)

emma
... -----> e
..e -----> m
.em -----> m
emm -----> a
mma -----> .
olivia
... -----> o
..o -----> l
.ol -----> i
oli -----> v
liv -----> i
ivi -----> a
via -----> .
ava
... -----> a
..a -----> v
.av -----> a
ava -----> .


In [6]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([16, 3]), torch.int64, torch.Size([16]), torch.int64)

### implementing the embedding lookup table

In [7]:
C = torch.randn((27,2))
C

tensor([[-1.8965, -0.0225],
        [ 1.4681, -1.4792],
        [ 0.8697, -1.7368],
        [ 0.0131,  0.4890],
        [ 1.3363,  1.6343],
        [ 0.0983, -0.8910],
        [-0.7017,  0.7597],
        [-1.6787,  2.1284],
        [-0.0212, -0.2836],
        [ 0.1510,  0.3850],
        [ 1.1204, -0.4448],
        [-2.0753,  0.2243],
        [-0.5967, -1.0707],
        [-0.8916, -0.4010],
        [-0.8383,  0.1286],
        [-0.4620, -1.0820],
        [ 0.5153,  0.7494],
        [-0.5258, -2.0552],
        [ 0.6360,  0.1904],
        [ 1.0642,  0.2889],
        [ 0.0911,  0.0809],
        [ 0.3670,  1.3864],
        [ 1.2662, -0.6361],
        [-0.3992, -0.7131],
        [ 0.7677, -0.4065],
        [-0.1272,  0.4585],
        [-1.2700, -0.1399]])

In [8]:
C[X]

tensor([[[-1.8965, -0.0225],
         [-1.8965, -0.0225],
         [-1.8965, -0.0225]],

        [[-1.8965, -0.0225],
         [-1.8965, -0.0225],
         [ 0.0983, -0.8910]],

        [[-1.8965, -0.0225],
         [ 0.0983, -0.8910],
         [-0.8916, -0.4010]],

        [[ 0.0983, -0.8910],
         [-0.8916, -0.4010],
         [-0.8916, -0.4010]],

        [[-0.8916, -0.4010],
         [-0.8916, -0.4010],
         [ 1.4681, -1.4792]],

        [[-1.8965, -0.0225],
         [-1.8965, -0.0225],
         [-1.8965, -0.0225]],

        [[-1.8965, -0.0225],
         [-1.8965, -0.0225],
         [-0.4620, -1.0820]],

        [[-1.8965, -0.0225],
         [-0.4620, -1.0820],
         [-0.5967, -1.0707]],

        [[-0.4620, -1.0820],
         [-0.5967, -1.0707],
         [ 0.1510,  0.3850]],

        [[-0.5967, -1.0707],
         [ 0.1510,  0.3850],
         [ 1.2662, -0.6361]],

        [[ 0.1510,  0.3850],
         [ 1.2662, -0.6361],
         [ 0.1510,  0.3850]],

        [[ 1.2662, -0